In [1]:
# 下载文件并命名为 Jamming_Classifier.zip
# !wget -O Jamming_Classifier.zip "https://zenodo.org/records/3783969/files/Jamming_Classifier.zip?download=1"

# 解压下载的 ZIP 文件
# !unzip Jamming_Classifier.zip

In [2]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 图像预处理：灰度图 → 256×256 → tensor (float32, 0~1)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # 强制灰度
    transforms.Resize((256, 256)),                # 缩放到统一大小
    transforms.ToTensor(),                        # 转换为 float32 & 归一化 [0,1]
])

# 设置路径（你原本是调反了，training 应该是训练集）
train_dir = 'Dataset/Jamming_Classifier/Image_training_database'
test_dir = 'Dataset/Jamming_Classifier/Image_testing_database'

# 构建 PyTorch Dataset（仅记录路径 & 标签，不加载数据）
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

# 构建 DataLoader：每次只加载一小批，避免内存溢出
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

# 获取类别名（ImageFolder 会自动按文件夹排序生成类别标签）
classes = train_dataset.classes
print("类别标签顺序:", classes)

类别标签顺序: ['DME', 'NB', 'NoJam', 'SingleAM', 'SingleChirp', 'SingleFM']


In [3]:
import os
# 统计每个类别的图像数量（保持与 ImageFolder 加载路径一致）
def count_images(folder_path, classes, dataset_name):
    print(f"\n{dataset_name} 圖片數量統計：")
    for label in classes:
        label_dir = os.path.join(folder_path, label)
        if not os.path.exists(label_dir):
            print(f"類別 {label} 不存在，跳過")
            continue
        count = len([file for file in os.listdir(label_dir) if file.endswith('.bmp')])
        print(f"{label}: {count} 張")

# 使用之前定义的路径和类名
count_images(train_dir, classes, '訓練集')
count_images(test_dir, classes, '測試集')


訓練集 圖片數量統計：
DME: 10000 張
NB: 10000 張
NoJam: 10000 張
SingleAM: 10000 張
SingleChirp: 10000 張
SingleFM: 10000 張

測試集 圖片數量統計：
DME: 10000 張
NB: 10000 張
NoJam: 10000 張
SingleAM: 10000 張
SingleChirp: 10000 張
SingleFM: 10000 張


In [7]:
import torch
import torch.nn as nn
from torchvision import models

# 使用预训练的 MobileNetV2 模型
base_model = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)

# 修改第一层卷积层（将输入通道数从 3 改为 1）
old_conv = base_model.conv1
new_conv = nn.Conv2d(
    in_channels=1,
    out_channels=old_conv.out_channels,
    kernel_size=old_conv.kernel_size,
    stride=old_conv.stride,
    padding=old_conv.padding,
    bias=old_conv.bias is not None
)

# 平均 RGB 权重 → 适配灰度
with torch.no_grad():
    new_conv.weight = nn.Parameter(old_conv.weight.mean(dim=1, keepdim=True))

base_model.conv1= new_conv

# 获取类别数（来自 ImageFolder）
num_classes = len(train_dataset.classes)

# 替换分类头
base_model.fc = nn.Linear(base_model.fc.in_features, num_classes)

# 冻结特征层参数（可选）
for param in base_model.parameters():
    param.requires_grad = False

for param in base_model.fc.parameters():
    param.requires_grad=True

# 显示模型结构确认
print(base_model)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [9]:
import torch.nn as nn
import torch.optim as optim

# 损失函数：适用于多分类任务
criterion = nn.CrossEntropyLoss()

# 只优化需要梯度更新的参数（即 requires_grad=True 的部分）
trainable_params = filter(lambda p: p.requires_grad, base_model.parameters())

# 优化器：只优化分类层
optimizer = optim.Adam(trainable_params, lr=0.001)

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model.to(device)


ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [10]:
from torch.utils.data import random_split, DataLoader
from torchvision import datasets

# 原始 ImageFolder 的完整训练集
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)

# 计算 80/20 分割
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_subset, val_subset = random_split(
    train_dataset,
    [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)

# 构建 DataLoader（分批加载）
train_loader = DataLoader(train_subset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_subset, batch_size=32, shuffle=False, num_workers=0)

# 测试集 DataLoader（不变）
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

# 类别标签（来自 ImageFolder 自动识别的 class 文件夹名）
classes = train_dataset.classes
print("类别顺序:", classes)

类别顺序: ['DME', 'NB', 'NoJam', 'SingleAM', 'SingleChirp', 'SingleFM']


In [11]:
epochs = 30
patience = 5
best_val_loss = float('inf')
best_state_dict = None
epochs_no_improve = 0

for epoch in range(1, epochs+1):
    base_model.train()  # 切换模型到训练模式
    train_loss_sum = 0.0
    train_correct = 0
    total_train = 0

    # 训练批次循环
    for X_batch, y_batch in train_loader:
        # 将数据加载到计算设备
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        # 前向传播
        # X_batch = X_batch.repeat(1, 3, 1, 1)  # 从灰度 (B,1,256,256) → RGB (B,3,256,256)
        outputs = base_model(X_batch)
        loss = criterion(outputs, y_batch)
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # 统计训练损失和准确度
        train_loss_sum += loss.item() * y_batch.size(0)
        _, pred_labels = torch.max(outputs, 1)
        train_correct += (pred_labels == y_batch).sum().item()
        total_train += y_batch.size(0)
    # 计算平均训练损失和准确率
    avg_train_loss = train_loss_sum / total_train
    train_accuracy = train_correct / total_train

    # 验证阶段
    base_model.eval()  # 切换模型到评估模式
    val_loss_sum = 0.0
    val_correct = 0
    total_val = 0
    # 在验证集上不需要计算梯度
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = base_model(X_batch)
            loss = criterion(outputs, y_batch)
            # 累计验证损失和准确度
            val_loss_sum += loss.item() * y_batch.size(0)
            _, pred_labels = torch.max(outputs, 1)
            val_correct += (pred_labels == y_batch).sum().item()
            total_val += y_batch.size(0)
    avg_val_loss = val_loss_sum / total_val
    val_accuracy = val_correct / total_val

    # 输出本轮训练的结果
    print(f"Epoch {epoch}/{epochs} - "
          f"loss: {avg_train_loss:.4f} - accuracy: {train_accuracy:.4f} - "
          f"val_loss: {avg_val_loss:.4f} - val_accuracy: {val_accuracy:.4f}")

    # Early Stopping 检查：若验证损失改善，则保存最佳模型权重
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_state_dict = base_model.state_dict()  # 保存当前最佳状态
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print("验证集 loss 多次没有改善，提前停止训练。")
            if best_state_dict is not None:
                base_model.load_state_dict(best_state_dict)  # 恢复最佳模型权重
            break

Epoch 1/30 - loss: 0.2192 - accuracy: 0.9292 - val_loss: 0.1321 - val_accuracy: 0.9530
Epoch 2/30 - loss: 0.1287 - accuracy: 0.9522 - val_loss: 0.1065 - val_accuracy: 0.9583
Epoch 3/30 - loss: 0.1127 - accuracy: 0.9576 - val_loss: 0.0971 - val_accuracy: 0.9653
Epoch 4/30 - loss: 0.1089 - accuracy: 0.9592 - val_loss: 0.0978 - val_accuracy: 0.9637
Epoch 5/30 - loss: 0.1032 - accuracy: 0.9605 - val_loss: 0.0921 - val_accuracy: 0.9666
Epoch 6/30 - loss: 0.0980 - accuracy: 0.9630 - val_loss: 0.0896 - val_accuracy: 0.9659
Epoch 7/30 - loss: 0.0952 - accuracy: 0.9631 - val_loss: 0.0823 - val_accuracy: 0.9700
Epoch 8/30 - loss: 0.0909 - accuracy: 0.9636 - val_loss: 0.0830 - val_accuracy: 0.9702
Epoch 9/30 - loss: 0.0933 - accuracy: 0.9636 - val_loss: 0.1198 - val_accuracy: 0.9564
Epoch 10/30 - loss: 0.0909 - accuracy: 0.9648 - val_loss: 0.0775 - val_accuracy: 0.9710
Epoch 11/30 - loss: 0.0875 - accuracy: 0.9659 - val_loss: 0.0769 - val_accuracy: 0.9713
Epoch 12/30 - loss: 0.0876 - accuracy: 0.

In [12]:
base_model.eval()  # 模型设为评估模式
test_correct = 0
total_test = 0

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        # X_batch = X_batch.repeat(1, 3, 1, 1)  # 转换为 RGB 三通道
        outputs = base_model(X_batch)
        _, pred_labels = torch.max(outputs, 1)
        test_correct += (pred_labels == y_batch).sum().item()
        total_test += y_batch.size(0)

test_accuracy = test_correct / total_test
print(f"测试集上的准确率: {test_accuracy * 100:.2f}%")


测试集上的准确率: 85.19%


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# 模型评估
base_model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        outputs = base_model(X_batch)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(y_batch.cpu().numpy())

# 计算评价指标
print("=== Classification Report ===")
print(classification_report(all_labels, all_preds, target_names=classes))

print("=== Confusion Matrix ===")
print(confusion_matrix(all_labels, all_preds))
